In [25]:
import requests
import pandas as pd

url = "http://user-segment.constellr.internal"

headers = {
    "accept": "application/json",
    "access_token": "test_api_key",
}

response = requests.get(f"{url}/areas-of-interest", headers=headers)

df = pd.DataFrame(response.json())
# gdf["geometry"] = gdf["area_of_interest"].apply(shape)
# gdf.crs = 4326

# Get all customers
response = requests.get(f"{url}/customers", headers=headers)
customers = response.json()
customers_df = pd.DataFrame(customers)
df = pd.merge(df, customers_df, on="customer_id", how="left")


aoi = df[df["name"] == "test-demo-bayer-emea-cluster1"].iloc[0].to_dict()

print(
    "SELECTED AOI:"
    "customer_id: " + aoi["customer_id"],
    "customer_name: " + aoi["customer_name"],
    "name: " + aoi["name"],
    sep="\n"
)

SELECTED AOI:customer_id: ead03d6d-89e5-42c9-bd55-df5ed1851270
customer_name: bayer-europe
name: test-demo-bayer-emea-cluster1


In [3]:
import json
import requests

ARGO_SERVER_ADDRESS = "http://argo.test.software.internal"

def place_order(namespace: str, workflow_template_name: str, order_params: dict):

    data = {
        "resourceKind": "WorkflowTemplate",
        "resourceName": workflow_template_name,
        "submitOptions": {
            "parameters": [f"{key}={value}" for key, value in order_params.items()]
        },
    }

    headers = {"Content-Type": "application/json"}
    submit_url = f"{ARGO_SERVER_ADDRESS}/api/v1/workflows/{namespace}/submit"

    response = requests.post(submit_url, json=data, headers=headers)
    response.raise_for_status()

    response_json = response.json()
    workflow_name = response_json["metadata"]["name"]
    return workflow_name


def print_workflow_url(namespace: str, workflow_name: str):
    print(f"{ARGO_SERVER_ADDRESS}/workflows/{namespace}/{workflow_name}")

In [ ]:
namespace = "lst30-pipeline"
workflow_template_name = "lst30-pipeline-v0.1.0"

order_params = {
    "start-datetime": "2023-07-24T00:00:00Z",
    "end-datetime": "2023-07-24T23:59:59Z",
    "area-of-interest": aoi["area_of_interest"],
    "maximum-cloud-cover": 75,
    "maximum-sun-elevation": 90,
    "minimum-sun-elevation": 10,
    "source": "landsat",  # or "ecostress", or "all"
    "atm-source": "ERA5",  # or CAMS
}

# the AOI must be serialized to a string
order_params["area-of-interest"] = json.dumps(order_params["area-of-interest"])

In [ ]:
workflow_name = place_order(namespace, workflow_template_name, order_params)
print_workflow_url(namespace, workflow_name)
print(workflow_name)